Text2Video-Zero

Paper:
https://arxiv.org/abs/2303.13439

GitHub:
https://github.com/Picsart-AI-Research/Text2Video-Zero

HuggingFace:
https://huggingface.co/docs/diffusers/api/pipelines/text_to_video_zero

## 1 generate

In [1]:
import torch
from diffusers import TextToVideoZeroPipeline, DPMSolverMultistepScheduler, DiffusionPipeline
import numpy as np
import imageio

2024-04-30 07:07:10.465809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 07:07:10.465873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 07:07:10.466941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 07:07:10.475791: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# model_id = "runwayml/stable-diffusion-v1-5"
model_id = "../Zoe_DreamBooth/weights_saved/2600"
pipe = TextToVideoZeroPipeline.from_pretrained(model_id, safety_checker=None, torch_dtype=torch.float16).to("cuda")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# memory optimization
pipe.enable_vae_slicing()

seed = 0
height = 400
width = 960
video_length = 24  #24 ÷ 4fps = 6 seconds
chunk_size = 8
motion_field_strength_x = 0
motion_field_strength_y = 0
prompt = "[A] banknote design, A Zen citrus tree surrounded by mechanical butterflies in a futuristic botanical garden, a citrus is in the middle giving a speech, (impressionist paintings), (((highly dynamic))), big movements, (watercolour style), blending of colours, soft gradient, Chinese painter Wu Guanzhong, Edward Hopper"
negative_prompt = "deformed, disfigured, poor details, bad anatomy"
num_inference_steps = 100

# Generate the video chunk-by-chunk
result = []
chunk_ids = np.arange(0, video_length, chunk_size - 1)
generator = torch.Generator(device="cuda")
for i in range(len(chunk_ids)):
    print(f"Processing chunk {i + 1} / {len(chunk_ids)}")
    ch_start = chunk_ids[i]
    ch_end = video_length if i == len(chunk_ids) - 1 else chunk_ids[i + 1]
    # Attach the first frame for Cross Frame Attention
    frame_ids = [0] + list(range(ch_start, ch_end))
    # Fix the seed for the temporal consistency
    generator.manual_seed(seed)
    output = pipe(
        prompt=prompt, 
        negative_prompt=negative_prompt, 
        num_inference_steps=num_inference_steps,
        height=height,
        width=width,
        motion_field_strength_x=motion_field_strength_x,
        motion_field_strength_y=motion_field_strength_y,
        video_length=len(frame_ids),
        generator=generator,
        frame_ids=frame_ids)
    result.append(output.images[1:])

# Concatenate chunks and save
result = np.concatenate(result)
result = [(r * 255).astype("uint8") for r in result]

imageio.mimsave("video_bn_1.mp4", result, fps=4)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.text_to_video_synthesis.pipeline_text_to_video_zero.TextToVideoZeroPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Processing chunk 1 / 4


  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/48 [00:00<?, ?it/s]

Processing chunk 4 / 4


  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

## 2 display video

In [7]:
import IPython
IPython.display.Video('video_bn_1.mp4', embed = True)